In [37]:
import geopandas as gpd
import pandas as pd

In [38]:
import ee
ee.Authenticate()  # Follow the instructions to authenticate
ee.Initialize()

In [39]:
shapefile = ee.FeatureCollection('projects/thesis-434321/assets/sy_merged')

In [40]:
shapefile.getInfo()

{'type': 'FeatureCollection',
 'columns': {'Origin': 'String',
  'group': 'Long',
  'plot1': 'String',
  'plot2': 'String',
  'plot3': 'String',
  'shape_P1': 'String',
  'shape_P2': 'String',
  'shape_P3': 'String',
  'sowing': 'String',
  'sowing_dat': 'String',
  'system:index': 'String'},
 'version': 1725331609050161,
 'id': 'projects/thesis-434321/assets/sy_merged',
 'properties': {'system:asset_size': 83814},
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[85.66504331012318, 25.901887944598492],
      [85.66519049501451, 25.901865671736882],
      [85.66528861804906, 25.90253006538257],
      [85.66513696615816, 25.902543469594285],
      [85.66513696615816, 25.90253896673495],
      [85.66504331012318, 25.901887944598492]]]},
   'id': '00000000000000000138',
   'properties': {'Origin': 'p1',
    'group': 108,
    'plot1': 'CT with 2 irrigations (21 DAS, 65 DAS)',
    'plot2': 'CT with 2 irrigations (21 DAS, 65 DAS)',
    'plot3': 'CT wi

In [41]:
def add_timeframe(feature):
    # Extract the sowing date from the feature
    sowing_date = ee.Date(feature.get('sowing_dat'))
    # print(sowing_date)
    
    # Calculate the start and end dates
    start_date = sowing_date.advance(-20, 'day')
    end_date = sowing_date.advance(150, 'day')
    
    # Return the feature with added time range properties
    return feature.set({'start_date': start_date, 'end_date': end_date})

In [42]:
shapefile = shapefile.map(add_timeframe)

In [43]:
# test if the function worked
shapefile.first().getInfo()['properties']['end_date']

{'type': 'Date', 'value': 1680912000000}

In [44]:
def clip_and_calculate_ndvi(image):
    # Clip the image to the shapefile
    image = image.clip(shapefile.geometry())
    
    # Calculate NDVI
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    
    # save time information
    date = image.date().format('YYYY-MM-dd')
    
    return image.addBands(ndvi).set('date', date)

In [45]:
def reduce_to_feature(img,feature):
    img_time = img.get('date')
    ndvi = img.select('NDVI')
    mean_ndvi = ndvi.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=feature.geometry(),
        scale=10  # Adjust scale depending on accuracy needs
        ).get('NDVI')
        
        # Return as two separate properties
    return ee.Feature(None, {'img_date': img_time, 'mean_ndvi': mean_ndvi})

In [46]:
def get_ndvi(feature):
    # Retrieve the start and end dates from the feature
    start = ee.Date(feature.get('start_date'))
    end = ee.Date(feature.get('end_date'))
    
    # Filter the Sentinel-2 image collection
    collection = ee.ImageCollection('COPERNICUS/S2') \
        .filterDate(start, end) \
        .filterBounds(feature.geometry()) \
        .map(clip_and_calculate_ndvi)
    
    # Aggregate results
    mean_ndvis = collection.map(lambda img: reduce_to_feature(img, feature)).aggregate_array('mean_ndvi')
    dates = collection.map(lambda img: reduce_to_feature(img, feature)).aggregate_array('img_date')
    
    return feature.set({'NDVI_data': mean_ndvis, 'dates': dates})

In [47]:
# Apply the NDVI calculation to each feature
ndvi_results = shapefile.map(get_ndvi)

In [48]:
ndvi_results.first().getInfo()

{'type': 'Feature',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[85.66504331012318, 25.901887944598492],
    [85.66519049501451, 25.901865671736882],
    [85.66528861804906, 25.90253006538257],
    [85.66513696615816, 25.902543469594285],
    [85.66513696615816, 25.90253896673495],
    [85.66504331012318, 25.901887944598492]]]},
 'id': '00000000000000000138',
 'properties': {'NDVI_data': [0.2596666466934959,
   0.29042583381664,
   0.19607808344718664,
   0.09330688104108736,
   0.0980638237864885,
   0.08244837284077393,
   0.07452026515035073,
   0.11293210451592976,
   0.11272438056944625,
   0.14331471731282952,
   0.012832078502893316,
   0.031628300024302024,
   0.0034079730647324567,
   0.0072246977662316185,
   0.027775292991915418,
   0.30830846475090673,
   0.27281887110029873,
   0.028025616827030517,
   0.39269952134671104,
   0.33516217970046863,
   0.3683603136004926,
   0.29850356389215815,
   0.2583195057563473,
   0.33203523749032887,
   0.31517526701692195,
  

In [50]:
# export the results
task = ee.batch.Export.table.toDrive(
    collection=ndvi_results,
    description='NDVI_export',
    fileFormat='CSV',
)

task.start()

In [49]:
task_test = ee.batch.Export.table.toDrive(
    collection=ndvi_results.limit(5),
    description='NDVI_export_test',
    fileFormat='CSV',
)

task_test.start()